In [28]:
from keras.models  import load_model
import keras.backend as K

import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer

from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

aspects = ['population', 'intervention', 'outcome']

In [2]:
pubmed_data = cPickle.load(open('../../preprocess/pubmed_vec_5000.p'))

In [5]:
file_model = '../store/weights/single_setup/TueOct1717:12:052017/loss.h5'
model = load_model(file_model)

In [8]:
model_aspect = {}
for aspect in aspects :
    model_aspect[aspect] = K.function(model.get_layer('pool_'+aspect).inputs + [K.learning_phase()], 
                          [model.get_layer('pool_'+aspect).get_output_at(0)])

In [9]:
pool_cdsr = {}
norm_pool_cdsr = {}
X_abs = pubmed_data.X

for aspect in aspects :
    pool_cdsr[aspect] = []
    for i in range(int(np.floor(len(X_abs)/1000))) :
        pool_cdsr[aspect].append(model_aspect[aspect]([X_abs[i*1000:(i+1)*1000,:], 0])[0])
    pool_cdsr[aspect] = np.concatenate(pool_cdsr[aspect])
    norm_pool_cdsr[aspect] = normalize(pool_cdsr[aspect], 'l2')

In [43]:
scores = {}
for aspect in aspects :
    #scores[aspect] = np.dot(norm_pool_cdsr[aspect], norm_pool_cdsr[aspect].T)
    scores[aspect] = cosine_similarity(pool_cdsr[aspect], pool_cdsr[aspect])
    np.fill_diagonal(scores[aspect], -1000)

In [ ]:
i, j = np.unravel_index(scores['outcome'].argmax(), scores['outcome'].shape)

In [54]:
print scores['intervention'][i, j]
i, j

7.84438


(604, 1933)

In [52]:
df = pd.read_csv('../../pubmed/pubmed_texts.csv')

In [53]:
list(df.iloc[[i,j]]['text'])

['The maximal double step length test can evaluate more adequately the decrease of physical function with age, than the maximal single step length test.\n\nThe maximal double step length (MDSL) test is a modified version of the maximal step length (MSL) test used to evaluate the dynamic balance related to falls in the elderly. Although this test can evaluate their physical function using movements similar to daily activities as compared to MSL, it has not been clarified whether it can adequately evaluate the activities of daily living (ADL) abilities of the elderly, as they decrease with age. This study aimed to examine the relationship between both MSL and MDSL on age and ADL in elderly subjects. Fifty-seven elderly females who live independently (age 74.8\xc2\xb15.6 years) were recruited. MSL and MDSL were conducted twice after completing an ADL questionnaire. Intra-class correlation coefficients (ICCs) of both tests were high (for MSL: ICC=0.95, for MDSL: ICC=0.81-0.82). Age and ADL